# CRC 校验码

- https://commandlinefanatic.com/cgi-bin/showarticle.cgi?article=art008

## 多项式计算

$$
{
    12x^5 + 23x^4 + 57x^3 + 79x^2 + 69x + 60
    \over 
    4x^3 + 5x^2 +  9x + 12
}
$$

In [6]:
# numerator 分子, denominator 分母
def divide_poly(num: list, den: list):
    qcount = len(num) - len(den) + 1
    quotient = [0 for _ in range(qcount)]

    for idx in range(qcount):
        nidx = len(num) - 1 - idx
        qidx = qcount - 1 - idx

        quotient[qidx] = num[nidx] // den[-1]

        for d in range(len(den)):
            num[qidx + d] -= den[d] * quotient[qidx]

    return quotient, num[: len(den) - 1]


num = [60.0, 69.0, 79.0, 57.0, 23.0, 12.0]
den = [12.0, 9.0, 5.0, 4.0]
divide_poly(num, den)


([5.0, 2.0, 3.0], [0.0, 0.0, 0.0])

## 二进制多项式除法

In [7]:
def divide_binary(num: list, den: list):
    qcount = len(num) - len(den) + 1
    quotient = [0 for _ in range(qcount)]

    for idx in range(qcount):
        nidx = len(num) - 1 - idx
        qidx = qcount - 1 - idx

        quotient[qidx] = (num[nidx] // den[-1]) % 2

        for d in range(len(den)):
            num[qidx + d] -= den[d] * quotient[qidx]
            num[qidx + d] = (num[qidx + d]) % 2

    return quotient, num[: len(den) - 1]

# 1101_0110_1100_00
num = [
    0.0, 0.0,
    0.0, 0.0, 1.0, 1.0,
    0.0, 1.0, 1.0, 0.0,
    1.0, 0.0, 1.0, 1.0,
]

den = [1.0, 1.0, 0.0, 0.0, 1.0]

divide_binary(num, den)


([0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 1.0, 1.0, 1.0])

## 计算 CRC32

In [8]:
import binascii

crc = binascii.crc32(b"ABC")
print(f"{crc:#x}")

crc = binascii.crc32(b"\x00")
print(f"{crc:#x}")

0xa3830348
0xd202ef8d


## 反转比特序列

实际上的字节序在物理上存储就是比特完全反转，但是一些关于字节序的描述给出了如下的例子：

- 大端：`0x12345678`
- 小段：`0x78563412`

这是由于，机器对小端字节序的解释，造成了这个现象；

In [9]:
def reverse_bits(value, width=8):
    string = b = "{:0{width}b}".format(value, width=width)
    return int(string[::-1], 2)


a = ord("A")
print(f"{a:08b} {reverse_bits(a):08b}")
print(f"{a:x} {reverse_bits(a):x}")


01000001 10000010
41 82


In [10]:
import numpy as np

den = [int(bit) for bit in f'{0x104C11DB7:b}']
den.reverse()

num = []
for ch in b"ABC\x00\x00\x00\x00": # 补足 CRC32 字节
    num.insert(0, [float(bit) for bit in f'{ch:08b}'])

num = np.array(num)
print(num.reshape(-1, 8))

quotient, remainder = divide_binary(num.reshape(-1), den)
crc = np.array(remainder)[::-1].reshape(-1, 4)

for c in crc:
    c = c.astype(np.int_)
    print(c, f"{c[3] + (c[2] << 1) + (c[1] << 2)  + (c[0] << 3):X}")

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 1.]
 [0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1.]]
[0 1 0 1] 5
[1 0 1 0] A
[0 1 0 1] 5
[1 0 1 1] B
[0 1 0 0] 4
[0 0 1 1] 3
[0 0 1 1] 3
[1 0 1 0] A


## 单个整型

In [11]:
def compute_crc0(data, len, divisor):
    while len > 0:
        len -= 1
        if data & 0x8000_0000:
            data = ((data << 1) ^ divisor) & 0xFFFF_FFFF
        else:
            data = (data << 1) & 0xFFFF_FFFF
    return data


divisor = 0x04C11DB7
# data = 0x8242C200
data = reverse_bits(0x434241, 32)
# print(f'{data:x}')
crc = compute_crc0(data, 3 * 8, divisor)
print(f"{crc:#x}")

crc = compute_crc0(0, 8, divisor)
print(f"{crc:#x}")

0x5a5b433a
0x0


## 多字节

In [12]:
def compute_crc1(data, len, divisor):
    crc = 0

    for i in range(len):
        crc = (crc ^ (reverse_bits(data[i]) << 24)) & 0xFFFF_FFFF

        for k in range(8):
            if crc & 0x8000_0000:
                crc = ((crc << 1) ^ divisor) & 0xFFFF_FFFF
            else:
                crc = (crc << 1) & 0xFFFF_FFFF

    return crc

divisor = 0x04C11DB7
data = b"ABC"

crc = compute_crc1(data, 3, divisor)
print(f"{crc:#x}")

crc = compute_crc0(0, 8, divisor)
print(f"{crc:#x}")

0x5a5b433a
0x0


你可能会有疑问，在一开始我们只处理了一个字节，但是后面以或操作却是整个 32 位校验和；这是由于异或的一些美好的性质：它具有交换律和结合律。

| a   | b   | c   |
| --- | --- | --- |
| 0   | 0   | 0   |
| 0   | 1   | 1   |
| 1   | 0   | 1   |
| 1   | 1   | 0   |

## ISO 的改进

上个实现直到输入的第一个 1 位才生效。这可能看起来不像一个重要的问题，但这意味着一个输入加上一串前导 0 不会被 CRC 计算检测为错误。

出于这个原因，官方的 CRC32 标准建议从全 1 开始，而不是 全 0；最后做取反操作；

- https://www.iso.org/standard/8559.html
- https://cdn.standards.iteh.ai/samples/8559/69473f113177492d9711be845cb6f992/ISO-3309-1991.pdf
- https://cdn.standards.iteh.ai/samples/8561/ee3e6fc1cc8641fabff5257e9660cf07/ISO-IEC-3309-1993.pdf

In [13]:
def compute_crc2(data, len, divisor):
    crc = 0xFFFF_FFFF

    for i in range(len):
        crc = (crc ^ (reverse_bits(data[i]) << 24)) & 0xFFFF_FFFF

        for k in range(8):
            if crc & 0x8000_0000:
                crc = ((crc << 1) ^ divisor) & 0xFFFF_FFFF
            else:
                crc = (crc << 1) & 0xFFFF_FFFF

    return reverse_bits((~crc) & 0xFFFF_FFFF, 32)


divisor = 0x04C11DB7
data = b"ABC"
crc = compute_crc2(data, 3, divisor)
print(f"{crc:#x}")
crc = compute_crc2(b"\x00", 1, divisor)
print(f"{crc:#x}")

0xa3830348
0xd202ef8d


## 小端字节序的优化

In [14]:
def compute_crc3(data, len, divisor):
    crc = 0xFFFFFFFF

    for i in range(len):
        crc = (crc ^ data[i]) & 0xFFFFFFFF
        for k in range(8):
            if crc & 1:
                crc = ((crc >> 1) ^ divisor) & 0xFFFFFFFF
            else:
                crc = (crc >> 1) & 0xFFFFFFFF
    return crc ^ 0xFFFFFFFF


# divisor = 0xEDB88320
divisor = reverse_bits(0x04C11DB7, 32)
data = b"ABC"
crc = compute_crc3(data, 3, divisor)
print(f"{crc:#x}")


0xa3830348


## 生成表

In [15]:
def crc_byte(byte, divisor):
    for k in range(8):
        if byte & 1:
            byte = ((byte >> 1) ^ divisor) & 0xFFFF_FFFF
        else:
            byte = (byte >> 1) & 0xFFFF_FFFF
    return byte


def compute_crc4(data, len, divisor):
    crc = 0xFFFFFFFF

    for i in range(len):
        crc = (crc_byte((crc ^ data[i]) & 0xFF, divisor) ^ (crc >> 8)) & 0xFFFF_FFFF

    return crc ^ 0xFFFFFFFF


divisor = 0xEDB88320
data = b"ABC"
crc = compute_crc4(data, 3, divisor)
print(f"{crc:#x}")

0xa3830348


In [16]:
def crc_byte(byte, divisor=0xEDB88320):
    for k in range(8):
        if byte & 1:
            byte = ((byte >> 1) ^ divisor) & 0xFFFF_FFFF
        else:
            byte = (byte >> 1) & 0xFFFF_FFFF
    return byte


crc_table = [crc_byte(i) for i in range(256)]


def compute_crc5(data, len):
    crc = 0xFFFFFFFF

    for i in range(len):
        crc = (crc_table[(crc ^ data[i]) & 0xFF] ^ (crc >> 8)) & 0xFFFF_FFFF

    return crc ^ 0xFFFFFFFF


data = b"ABC"
crc = compute_crc5(data, 3)
print(f"{crc:#x}")

0xa3830348
